In [1]:
import pandas as pd
import rdflib as rdf
from SPARQLWrapper import SPARQLWrapper, JSON
import math
import numpy as np
import re

In [ ]:
df_bx = pd.read_csv('/home/ignacio/Datasets/BX-CSV-Dump/BX-Book-Ratings.csv', sep=';', encoding='iso-8859-15')
df_bx.head()

In [ ]:
df_bx_users = pd.read_csv('/home/ignacio/Datasets/BX-CSV-Dump/BX-Users.csv', sep=';', encoding='iso-8859-15')
df_bx_users.head()

In [ ]:
df_bx_book = pd.read_csv('/home/ignacio/Datasets/BX-CSV-Dump/BX-Books.csv', sep=';', encoding='iso-8859-15', error_bad_lines=False)
df_bx_book.head()

In [ ]:
df_author = df_bx_book['Book-Author']

In [ ]:
df_author.head()

In [ ]:
len(df_author)

In [ ]:
df_author = df_author.drop_duplicates()

In [ ]:
len(df_author)

In [ ]:
df_author=df_author.str.lower()

In [ ]:
df_author = df_author.drop_duplicates()

In [ ]:
len(df_author)

In [ ]:
df_author = df_author.str.title()

In [ ]:
df_author.head(50)

In [ ]:
df_author.to_csv('/home/ignacio/Datasets/BX-CSV-Dump/BX-Author.csv', index=False)

In [ ]:
df_bx_book[df_bx_book['Book-Author'].str.contains('Italo Calvino')]

## Get author influence information from dbpedia

#### Clean dataset (drop duplicate values)

In [2]:
df_authors = pd.read_csv('/home/ignacio/Datasets/BX-CSV-Dump/BX-Author.csv')
df_authors.head(25)

,Authors,Authors_mod
0,Mark P. O. Morford,Mark P.+ O.+ Morford
1,Richard Bruce Wright,Richard Bruce Wright
2,Carlo D'Este,Carlo D'Este
3,Gina Bari Kolata,Gina Bari Kolata
4,Amy Tan,Amy Tan
5,Robert Cowley,Robert Cowley
6,Scott Turow,Scott Turow
7,David Cordingly,David Cordingly
8,Ann Beattie,Ann Beattie
9,David Adams Richards,David Adams Richards


In [3]:
df_authors_abr = df_authors['Authors'].str.findall(r'^([A-Z](\.)?\s)+[A-Z][a-z]+$')
df_authors_abr = df_authors_abr[df_authors_abr.str.len() > 0]
df_authors_abr.head(10)

Series([], Name: Authors, dtype: object)

In [ ]:
df_authors = df_authors['Authors'].str.replace(r'[\\\"]', '')

In [ ]:
df_authors['Authors'] = df_authors['Authors'].str.replace(r'[\[\]]', '')
df_authors['Authors_mod'] = df_authors['Authors_mod'].str.replace(r'[\[\]]', '')

In [ ]:
df_authors.head()

In [ ]:
df_authors_abr.iloc[0]

In [ ]:
len(df_authors_abr)

In [ ]:
df_authors = df_authors.drop(list(df_authors_abr.index))
df_authors.head(25)

In [ ]:
len(df_authors)

In [ ]:
df_aux = pd.DataFrame(data= df_authors.values, columns=['Authors'])
df_authors = df_aux
df_authors.head()

In [ ]:
df_authors['Authors_mod'] = df_authors['Authors'].str.replace(r'\.\s', '.+ ')
df_authors.head()

In [ ]:
df_authors = df_authors.drop([94221])
df_authors.head()

In [ ]:
df_authors.to_csv('/home/ignacio/Datasets/BX-CSV-Dump/BX-Author.csv', index=False)

#### BDpedia query

In [ ]:
sparql = SPARQLWrapper("http://dbpedia.org/sparql")

In [ ]:
def find_influence( row ):
    
    sparql = SPARQLWrapper("http://dbpedia.org/sparql")
    
    stm = """
        SELECT DISTINCT ?Aname 
        where {
          ?A dbo:influenced ?B.
          ?A a dbo:Person.
          ?B a dbo:Person.
          ?B foaf:name ?Bname .
          ?A foaf:name ?Aname 
          FILTER(regex(?Bname, \""""+ row['Authors_mod'] +"""\"))
         }"""
         
    sparql.setQuery(stm)
    sparql.setReturnFormat(JSON)
    results = sparql.query().convert()
    director = []
    for result in results["results"]["bindings"]:
        director.append(result["Aname"]["value"])
    return director

In [ ]:
frame = 1000
max_complete_frames = math.floor(len(df_authors)/frame)
max_complete_frames

In [ ]:
for i in np.arange(94, max_complete_frames):
    df_test = df_authors.loc[i*frame : (i+1)*frame]
    df_test['influenced'] = df_test.apply(find_influence, axis=1)
    df_test.to_csv('/home/ignacio/Datasets/BX-CSV-Dump/BX-authors-influence_'+ str(i) +'.csv', index=False)
    print("Iteration " + str(i) )


In [ ]:
if len(df_authors) % frame != 0:
    df_test = df_authors.loc[max_complete_frames*frame : len(df_authors) ]
    df_test['influenced'] = df_test.apply(find_influence, axis=1)
    df_test.to_csv('/home/ignacio/Datasets/BX-CSV-Dump/BX-authors-influence_'+ str(max_complete_frames ) +'.csv', index=False)

In [ ]:
df_authors_influence = pd.read_csv('/home/ignacio/Datasets/BX-CSV-Dump/BX-authors-influence_0.csv')

for i in range(max_complete_frames+1):
    df_aux = pd.read_csv('/home/ignacio/Datasets/BX-CSV-Dump/BX-authors-influence_'+ str(i) +'.csv')
    df_authors_influence = pd.concat([df_authors_influence, df_aux], ignore_index=True)
    
df_authors_influence.head()

In [ ]:
df_authors_influence = df_authors_influence.drop(['Authors_mod'], axis=1)
df_authors_influence.head()

In [ ]:
df_authors_influence['influenced'] = df_authors_influence['influenced'].str.replace(r'[\[\]\'\']', '')
df_authors_influence = df_authors_influence[df_authors_influence['influenced'].str.len() > 0]
df_authors_influence.head()

In [ ]:
len(df_authors_influence)

In [ ]:
df_authors_influence.to_csv('/home/ignacio/Datasets/BX-CSV-Dump/BX-authors-influence-edge.csv', index=False)

## Clean duplicates author last-name

In [ ]:
df_authors_edge = pd.read_csv('/home/ignacio/Datasets/BX-CSV-Dump/BX-authors-influence-edge.csv')
df_authors_edge.head()

In [ ]:
len(df_authors_edge)

In [ ]:
df_authors_edge = df_authors_edge.drop_duplicates(subset=['Authors'])
len(df_authors_edge)

In [ ]:
s_authors = df_authors_edge['Authors']
s_authors.head()

In [ ]:
s_authors = s_authors.drop_duplicates()
len(s_authors)

In [ ]:
df_authors_edge.to_csv('/home/ignacio/Datasets/BX-CSV-Dump/BX-authors-influence-edge.csv', index=False)

In [ ]:
def find_duplicate_last_name(row, df):
    regx = re.compile('.+' + row + '$')
    df_aux = df[df['Authors'].str.contains(regx)]
    if len(df_aux) == 0 :
        return False
    else:
        return True
    

In [ ]:
author_last_name = df_authors_edge['Authors'].str.findall(r'^[A-Z][a-z]+$')
author_last_name = author_last_name[ author_last_name.str.len() > 0]
author_last_name.head()

In [ ]:
author_last_name = author_last_name.apply(lambda x : x[0])

In [ ]:
author_last_name.head()

In [ ]:
duplicates = author_last_name.apply(find_duplicate_last_name, df=df_authors_edge)
duplicates.head()

In [ ]:
len(duplicates)

In [ ]:
duplicates = duplicates[duplicates == True ]
duplicates.head()

In [ ]:
df_authors_edge = df_authors_edge.drop(duplicates.index)
df_authors_edge.head(30)

In [ ]:
df_authors_edge.to_csv('/home/ignacio/Datasets/BX-CSV-Dump/BX-authors-influence-edge.csv', index=False)

In [ ]:
regx = re.compile('.+' + author_last_name.iloc[0] + '$')
regx

In [ ]:
df_aux = df_authors_edge[df_authors_edge['Authors'].str.contains(regx)]

In [ ]:
len(df_aux)

In [ ]:
df_test = df_authors.loc[[497]]
df_test

In [ ]:
df_baudelaire = s_authors.str.findall(r'.*Stuart$')
df_baudelaire = df_baudelaire[df_baudelaire.str.len() > 0]
df_baudelaire

In [ ]:
s_list = list(df_authors_edge.loc[[497]]['influenced'])

In [ ]:
s_list = s_list[0].split(r', ')

In [ ]:
len(s_list)

In [ ]:
df_test['influenced'] = df_test.apply(find_influence, axis=1)
df_test